In [1]:
import pandas as pd


path = "results/aime_1983_2023_qwq-32b_traces.csv"
df = pd.read_csv(path)
df

,ID,Year,Problem Number,Question,Answer,Part,Reasoning,Solution Attempt,Answer Attempt,Correct
0,1983-3,1983,3,What is the product of the real roots of the e...,20,NaN,"Alright, so I need to solve the equation \( x^...",Let \( y = x^2 + 18x + 30 \). The given equati...,20.0,True
1,1983-1,1983,1,"Let $x$ , $y$ and $z$ all exceed $1$ and let $...",60,NaN,"Okay, so I need to find the value of log base ...","Given the equations \(\log_x w = 24\), \(\log_...",60.0,True
2,1983-2,1983,2,"Let $f(x)=|x-p|+|x-15|+|x-p-15|$ , where $0 < ...",15,NaN,"Okay, so I need to find the minimum value of t...",To determine the minimum value of the function...,15.0,True
3,1983-6,1983,6,Let $a_n=6^{n}+8^{n}$ . Determine the remainde...,35,NaN,"Okay, so I need to find the remainder when a_{...",To determine the remainder when \(a_{83} = 6^{...,35.0,True
4,1983-7,1983,7,Twenty five of King Arthur's knights are seate...,57,NaN,"Okay, so I need to find the probability that w...",To find the probability that at least two of t...,57.0,True
...,...,...,...,...,...,...,...,...,...,...
914,2019-I-13,2019,13,"Triangle $ABC$ has side lengths $AB=4$ , $BC=5...",32,I,"Okay, so I have this geometry problem here, an...","Given triangle \(ABC\) with \(AB = 4\), \(BC =...",32.0,True
915,2022-II-15,2022,15,Two externally tangent circles $\omega_1$ and ...,140,II,"Okay, so I need to find the area of the convex...",Given two externally tangent circles \(\omega_...,240.0,False
916,2018-II-14,2018,14,The incircle $\omega$ of triangle $ABC$ is tan...,227,II,"Okay, so I have this geometry problem here, an...",Given triangle \(ABC\) with incircle \(\omega\...,19.0,False
917,2011-I-14,2011,14,Let $A_1 A_2 A_3 A_4 A_5 A_6 A_7 A_8$ be a reg...,37,I,"Okay, so I need to solve this problem about a ...",Given a regular octagon \( A_1 A_2 A_3 A_4 A_5...,4.0,False


In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-32B-Instruct")


def tokenize_text(question, reasoning, attempt):
    attempt = "Answer: \\boxed{" + str(attempt) + "}"
    question = question.strip()
    question += '\nPlease reason step by step. Format your final answer as follows: "Answer: \\boxed{answer}". Don\'t include any other text.'
    text = tokenizer.apply_chat_template(
        [
            {"role": "user", "content": question.strip()},
            {
                "role": "assistant",
                "content": "<|im_start|>think\n"
                + reasoning.strip()
                + "\n<|im_start|>answer\n"
                + attempt.strip(),
            },
        ],
        tokenize=False,
    )
    return text


# Apply chat template to all rows in the dataframe
df["templated_response"] = df.apply(
    lambda row: tokenize_text(
        row["Question"],
        row["Reasoning"],
        str(int(row["Answer Attempt"])),
    ),
    axis=1,
)

df

,ID,Year,Problem Number,Question,Answer,Part,Reasoning,Solution Attempt,Answer Attempt,Correct,templated_response
0,1983-3,1983,3,What is the product of the real roots of the e...,20,NaN,"Alright, so I need to solve the equation \( x^...",Let \( y = x^2 + 18x + 30 \). The given equati...,20.0,True,"<|im_start|>system\nYou are Qwen, created by A..."
1,1983-1,1983,1,"Let $x$ , $y$ and $z$ all exceed $1$ and let $...",60,NaN,"Okay, so I need to find the value of log base ...","Given the equations \(\log_x w = 24\), \(\log_...",60.0,True,"<|im_start|>system\nYou are Qwen, created by A..."
2,1983-2,1983,2,"Let $f(x)=|x-p|+|x-15|+|x-p-15|$ , where $0 < ...",15,NaN,"Okay, so I need to find the minimum value of t...",To determine the minimum value of the function...,15.0,True,"<|im_start|>system\nYou are Qwen, created by A..."
3,1983-6,1983,6,Let $a_n=6^{n}+8^{n}$ . Determine the remainde...,35,NaN,"Okay, so I need to find the remainder when a_{...",To determine the remainder when \(a_{83} = 6^{...,35.0,True,"<|im_start|>system\nYou are Qwen, created by A..."
4,1983-7,1983,7,Twenty five of King Arthur's knights are seate...,57,NaN,"Okay, so I need to find the probability that w...",To find the probability that at least two of t...,57.0,True,"<|im_start|>system\nYou are Qwen, created by A..."
...,...,...,...,...,...,...,...,...,...,...,...
914,2019-I-13,2019,13,"Triangle $ABC$ has side lengths $AB=4$ , $BC=5...",32,I,"Okay, so I have this geometry problem here, an...","Given triangle \(ABC\) with \(AB = 4\), \(BC =...",32.0,True,"<|im_start|>system\nYou are Qwen, created by A..."
915,2022-II-15,2022,15,Two externally tangent circles $\omega_1$ and ...,140,II,"Okay, so I need to find the area of the convex...",Given two externally tangent circles \(\omega_...,240.0,False,"<|im_start|>system\nYou are Qwen, created by A..."
916,2018-II-14,2018,14,The incircle $\omega$ of triangle $ABC$ is tan...,227,II,"Okay, so I have this geometry problem here, an...",Given triangle \(ABC\) with incircle \(\omega\...,19.0,False,"<|im_start|>system\nYou are Qwen, created by A..."
917,2011-I-14,2011,14,Let $A_1 A_2 A_3 A_4 A_5 A_6 A_7 A_8$ be a reg...,37,I,"Okay, so I need to solve this problem about a ...",Given a regular octagon \( A_1 A_2 A_3 A_4 A_5...,4.0,False,"<|im_start|>system\nYou are Qwen, created by A..."


In [10]:
print(df.iloc[0]["templated_response"])

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the product of the real roots of the equation $x^2 + 18x + 30 = 2 \sqrt{x^2 + 18x + 45}$ ?
Please reason step by step. Format your final answer as follows: "Answer: \boxed{answer}". Don't include any other text.<|im_end|>
<|im_start|>assistant
<|im_start|>think
Alright, so I need to solve the equation \( x^2 + 18x + 30 = 2 \sqrt{x^2 + 18x + 45} \) and find the product of its real roots. Hmm, okay. Let me see how to approach this. 

First, the equation has a square root on one side, which might complicate things. Maybe I can simplify it by making a substitution. Let me think... If I let \( y = x^2 + 18x + 30 \), then the equation becomes \( y = 2 \sqrt{y + 15} \), since the expression under the square root is \( x^2 + 18x + 45 \), which is 15 more than \( y \). That might work. Let me write that down:

Let \( y = x^2 + 18x + 30 \). Then the equation becomes:
\[
y = 

In [11]:
# Push the dataframe to the Hugging Face Hub in the train split
from huggingface_hub import HfApi
import os
from tempfile import NamedTemporaryFile
from datasets import Dataset

# Convert pandas DataFrame to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)

# Create a DatasetDict with a train split
from datasets import DatasetDict

dataset_dict = DatasetDict({"train": hf_dataset})

# Save the dataset to a temporary directory
with NamedTemporaryFile(suffix=".csv", delete=False) as temp_file:
    df.to_csv(temp_file.name, index=False)
    temp_file_path = temp_file.name

# Initialize the Hugging Face API client
api = HfApi()

# Get the API token from environment variable
token = os.environ.get("HUGGINGFACE_CLI")

# Define repository details
repo_id = "jonathanyin/aime_1983_2023_qwq-32b_traces"
repo_type = "dataset"

# Push the dataset to the Hub
dataset_dict.push_to_hub(repo_id=repo_id, token=token)

# Clean up the temporary file
os.unlink(temp_file_path)

print(f"Successfully pushed dataframe to {repo_id} in the train split")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Successfully pushed dataframe to jonathanyin/aime_1983_2023_qwq-32b_traces in the train split
